<a href="https://colab.research.google.com/github/vedantjhunjhunwal/sodio-vedant-task/blob/main/Untitled13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

done this using selinium for good accuracy

In [2]:
!pip install selenium
!pip install webdriver-manager
import os
import json
import sys
import logging
import subprocess
from datetime import datetime
from urllib.parse import urljoin, urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class ChromeInstaller:
    @staticmethod
    def install_chrome():
        """Install Chrome browser if not present"""
        try:
            if sys.platform.startswith('linux'):
                logger.info("Installing Google Chrome...")
                subprocess.run([
                    'wget',
                    'https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb'
                ], check=True)
                subprocess.run([
                    'sudo', 'apt', 'install', '-y',
                    './google-chrome-stable_current_amd64.deb'
                ], check=True)
                subprocess.run([
                    'rm', 'google-chrome-stable_current_amd64.deb'
                ], check=True)
                logger.info("Chrome installed successfully")
            else:
                logger.warning("Automatic Chrome installation only supported on Linux")
        except subprocess.CalledProcessError as e:
            logger.error(f"Failed to install Chrome: {e}")
            sys.exit(1)

class UIExtractor:
    def __init__(self):
        self.driver = self._init_driver()

    def _init_driver(self):
        """Initialize Selenium WebDriver with automatic Chrome installation"""
        try:
            options = Options()
            options.add_argument('--headless')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')

            try:
                service = Service(ChromeDriverManager().install())
                driver = webdriver.Chrome(service=service, options=options)
                driver.implicitly_wait(5)
                return driver
            except Exception as e:
                if "cannot find Chrome binary" in str(e):
                    ChromeInstaller.install_chrome()
                    service = Service(ChromeDriverManager().install())
                    driver = webdriver.Chrome(service=service, options=options)
                    driver.implicitly_wait(5)
                    return driver
                raise
        except Exception as e:
            logger.error(f"Failed to initialize WebDriver: {str(e)}")
            sys.exit(1)

    def validate_url(self, url):
        """Validate and normalize the URL"""
        try:
            result = urlparse(url)
            if not all([result.scheme, result.netloc]):
                raise ValueError
            return url if url.startswith(('http://', 'https://')) else f'https://{url}'
        except:
            return None

    def extract_ui_elements(self, url):
        """Extract all UI elements from the webpage"""
        try:
            self.driver.get(url)
            base_url = url

            results = {
                'buttons': [],
                'links': [],
                'inputs': [],
                'forms': [],
                'metadata': {
                    'url': url,
                    'timestamp': datetime.now().isoformat(),
                    'domain': urlparse(url).netloc,
                    'title': self.driver.title
                }
            }

            # Extract buttons
            self._extract_buttons(results, base_url)

            # Extract links
            self._extract_links(results, base_url)

            # Extract inputs (including textareas and selects)
            self._extract_inputs(results)

            # Extract forms
            self._extract_forms(results, base_url)

            # Update counts
            results['metadata'].update({
                'buttons_count': len(results['buttons']),
                'links_count': len(results['links']),
                'inputs_count': len(results['inputs']),
                'forms_count': len(results['forms'])
            })

            return results

        except WebDriverException as e:
            logger.error(f"Selenium error: {str(e)}")
            return None
        except Exception as e:
            logger.error(f"Extraction error: {str(e)}")
            return None

    def _extract_buttons(self, results, base_url):
        """Extract all button elements"""
        # Standard buttons
        buttons = self.driver.find_elements(By.TAG_NAME, 'button')
        for btn in buttons:
            results['buttons'].append(self._create_button_dict(btn))

        # Input buttons
        input_buttons = self.driver.find_elements(
            By.CSS_SELECTOR, 'input[type="button"], input[type="submit"], input[type="reset"]')
        for inp in input_buttons:
            results['buttons'].append(self._create_input_button_dict(inp))

        # Link buttons
        link_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'a[class*="btn"]')
        for a in link_buttons:
            results['buttons'].append(self._create_link_button_dict(a, base_url))

    def _create_button_dict(self, element):
        """Create dictionary for button element"""
        return {
            'element': 'button',
            'text': element.text.strip(),
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'name': element.get_attribute('name'),
                'disabled': element.get_attribute('disabled') is not None,
                'onclick': element.get_attribute('onclick')
            }
        }

    def _create_input_button_dict(self, element):
        """Create dictionary for input button"""
        return {
            'element': f'input_{element.get_attribute("type")}',
            'value': element.get_attribute('value'),
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'name': element.get_attribute('name'),
                'disabled': element.get_attribute('disabled') is not None,
                'onclick': element.get_attribute('onclick')
            }
        }

    def _create_link_button_dict(self, element, base_url):
        """Create dictionary for link button"""
        return {
            'element': 'link_button',
            'text': element.text.strip(),
            'href': urljoin(base_url, element.get_attribute('href')),
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'target': element.get_attribute('target'),
                'onclick': element.get_attribute('onclick')
            }
        }

    def _extract_links(self, results, base_url):
        """Extract all link elements"""
        links = self.driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if not href:
                continue

            results['links'].append({
                'text': link.text.strip(),
                'href': urljoin(base_url, href),
                'id': link.get_attribute('id'),
                'classes': link.get_attribute('class'),
                'attributes': {
                    'target': link.get_attribute('target'),
                    'title': link.get_attribute('title'),
                    'rel': link.get_attribute('rel')
                }
            })

    def _extract_inputs(self, results):
        """Extract all input elements (including textareas and selects)"""
        # Standard inputs
        input_types = [
            'text', 'password', 'email', 'number', 'tel', 'url',
            'search', 'date', 'file', 'checkbox', 'radio', 'range',
            'color', 'hidden'
        ]

        for input_type in input_types:
            inputs = self.driver.find_elements(By.CSS_SELECTOR, f'input[type="{input_type}"]')
            for inp in inputs:
                results['inputs'].append(self._create_input_dict(inp, input_type))

        # Textareas
        textareas = self.driver.find_elements(By.TAG_NAME, 'textarea')
        for textarea in textareas:
            results['inputs'].append(self._create_textarea_dict(textarea))

        # Selects
        selects = self.driver.find_elements(By.TAG_NAME, 'select')
        for select in selects:
            results['inputs'].append(self._create_select_dict(select))

    def _create_input_dict(self, element, input_type):
        """Create dictionary for input element"""
        return {
            'element': 'input',
            'type': input_type,
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'name': element.get_attribute('name'),
                'placeholder': element.get_attribute('placeholder'),
                'value': element.get_attribute('value'),
                'required': element.get_attribute('required') is not None,
                'disabled': element.get_attribute('disabled') is not None,
                'readonly': element.get_attribute('readonly') is not None
            }
        }

    def _create_textarea_dict(self, element):
        """Create dictionary for textarea element"""
        return {
            'element': 'textarea',
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'name': element.get_attribute('name'),
                'placeholder': element.get_attribute('placeholder'),
                'rows': element.get_attribute('rows'),
                'cols': element.get_attribute('cols'),
                'required': element.get_attribute('required') is not None,
                'disabled': element.get_attribute('disabled') is not None,
                'value': element.get_attribute('value')
            }
        }

    def _create_select_dict(self, element):
        """Create dictionary for select element"""
        options = []
        for opt in element.find_elements(By.TAG_NAME, 'option'):
            options.append({
                'text': opt.text.strip(),
                'value': opt.get_attribute('value'),
                'selected': opt.get_attribute('selected') is not None
            })

        return {
            'element': 'select',
            'id': element.get_attribute('id'),
            'classes': element.get_attribute('class'),
            'attributes': {
                'name': element.get_attribute('name'),
                'required': element.get_attribute('required') is not None,
                'disabled': element.get_attribute('disabled') is not None,
                'options_count': len(options),
                'options': options
            }
        }

    def _extract_forms(self, results, base_url):
        """Extract all form elements"""
        forms = self.driver.find_elements(By.TAG_NAME, 'form')
        for form in forms:
            form_inputs = form.find_elements(By.CSS_SELECTOR, 'input, textarea, select, button')

            input_types = list(set(
                inp.get_attribute('type') if inp.tag_name == 'input' else inp.tag_name
                for inp in form_inputs
            ))

            results['forms'].append({
                'id': form.get_attribute('id'),
                'classes': form.get_attribute('class'),
                'attributes': {
                    'action': urljoin(base_url, form.get_attribute('action') or base_url),
                    'method': (form.get_attribute('method') or 'get').upper(),
                    'name': form.get_attribute('name'),
                    'enctype': form.get_attribute('enctype'),
                    'inputs_count': len(form_inputs),
                    'input_types': input_types
                }
            })

    def save_results(self, output_dir="output"):
        """Save results to elements.json"""
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, "elements.json")

        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(self.results, f, indent=2, ensure_ascii=False)
            logger.info(f"Results saved to {output_file}")
            return True
        except Exception as e:
            logger.error(f"Failed to save results: {str(e)}")
            return False

    def close(self):
        """Clean up resources"""
        try:
            if hasattr(self, 'driver') and self.driver:
                self.driver.quit()
        except Exception as e:
            logger.error(f"Error closing driver: {str(e)}")

def main():
    """Main execution function"""
    print("=== Web UI Element Extractor (Selenium) ===")

    extractor = None
    try:
        extractor = UIExtractor()

        # Get URL from user
        while True:
            url = input("Enter website URL (e.g., https://demoblaze.com): ").strip()
            validated_url = extractor.validate_url(url)
            if validated_url:
                break
            print("Invalid URL. Please include http:// or https://")

        logger.info(f"Starting extraction from: {validated_url}")

        # Extract UI elements
        extractor.results = extractor.extract_ui_elements(validated_url)
        if not extractor.results:
            logger.error("No data extracted")
            return

        # Save results
        if extractor.save_results():
            print("\n=== Extraction Successful ===")
            print(f"• Buttons found: {extractor.results['metadata']['buttons_count']}")
            print(f"• Links found: {extractor.results['metadata']['links_count']}")
            print(f"• Inputs found: {extractor.results['metadata']['inputs_count']}")
            print(f"• Forms found: {extractor.results['metadata']['forms_count']}")
            print(f"\nResults saved to elements.json in /output folder")
        else:
            logger.error("Failed to save results")

    except Exception as e:
        logger.error(f"Fatal error: {str(e)}")
    finally:
        if extractor:
            extractor.close()

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 24.3 MB/s eta 0:00:00
=== Web UI Element Extractor (Selenium) ===
Enter website URL (e.g., https://demoblaze.com): https://demoblaze.com

=== Extraction Successful ===
• Buttons found: 28
• Links found: 32
• Inputs found: 16
• Forms found: 5

Results saved to elements.json in /output folder
